In [137]:
%matplotlib inline
import pandas as pd
import numpy as np
import re
from matplotlib.ticker import MaxNLocator
import matplotlib.pyplot as plt
from requests import get
from bs4 import BeautifulSoup
import time

In [138]:
url = 'https://en.wikipedia.org/wiki/Fuel_economy_in_aircraft'
r = get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [158]:
dataframes = []
for table in soup.find_all("table"):
    table_headers = []
    dict_plane = {}
    for header in table.find_all("th"):
        head = header.text.replace("\n", "")
        table_headers.append(head)
        dict_plane[head] = []
    for line in table.find_all("tr"):
        for i, elem in enumerate(line.find_all('td')):
            dict_plane[table_headers[i]].append(elem.text.replace("\n", ""))
    dataframes.append(dict_plane)
    
dataframes = dataframes[:-1] #We don't want the private jets
print("There are " + str(len(dataframes))+ " dataframes")

There are 5 dataframes


In [159]:
commuter = pd.DataFrame.from_dict(dataframes[0]) # 560km
regional = pd.DataFrame.from_dict(dataframes[1]) # 926km - 1267km
short_haul = pd.DataFrame.from_dict(dataframes[2])# 1900km
medium_haul = pd.DataFrame.from_dict(dataframes[3]) # 3240km - 6300km
long_haul = pd.DataFrame.from_dict(dataframes[4]) # 8610km - 13330km

In [160]:
commuter

,Model,First flight,Seats,Fuel burn,Fuel per seat
0,Antonov An-148 (241 nmi),2004,89,4.23 kg/km (15.0 lb/mi),5.95 L/100 km (39.5 mpg‑US)[33]
1,Antonov An-158 (241 nmi),2010,99,4.34 kg/km (15.4 lb/mi),5.47 L/100 km (43.0 mpg‑US)[33]
2,ATR 42-500,1995,48,1.26 kg/km (4.5 lb/mi),3.15 L/100 km (75 mpg‑US)[34]
3,ATR 72-500,1997,70,1.42 kg/km (5.0 lb/mi),2.53 L/100 km (93 mpg‑US)[34]
4,Beechcraft 1900D (226 nm),1982,19,1.00 kg/km (3.56 lb/mi),6.57 L/100 km (35.8 mpg‑US)[35]
5,Bombardier CRJ100,1991,50,2.21 kg/km (7.83 lb/mi),5.50 L/100 km (42.8 mpg‑US)[36]
6,Bombardier CRJ200,1995,50,2.18 kg/km (7.73 lb/mi),5.43 L/100 km (43.3 mpg‑US)[36]
7,Bombardier CRJ700,1999,70,2.95 kg/km (10.47 lb/mi),5.25 L/100 km (44.8 mpg‑US)[36]
8,Bombardier CRJ900,2001,88,3.47 kg/km (12.31 lb/mi),4.91 L/100 km (47.9 mpg‑US)[36]
9,Bombardier Dash 8 Q400,1998,78,2.16 kg/km (7.7 lb/mi),3.46 L/100 km (68.0 mpg‑US)[37]


In [200]:
regional.columns = ["Model", "First flight", "Seats", "Sector", "Fuel burn", "Fuel per seat"]
regional

,Model,First flight,Seats,Sector,Fuel burn,Fuel per seat
0,Airbus A319neo,2015,144,"600 nmi (1,100 km)",3.37 kg/km (11.94 lb/mi),2.92 L/100 km (80.6 mpg‑US)[45]
1,Airbus A319neo,2015,124,"660 nmi (1,220 km)",2.82 kg/km (10 lb/mi),2.82 L/100 km (83.5 mpg‑US)[46]
2,Airbus A320neo,2015,154,"660 nmi (1,220 km)",2.79 kg/km (9.9 lb/mi),2.25 L/100 km (104.7 mpg‑US)[46]
3,Airbus A321neo,2015,192,"660 nmi (1,220 km)",3.30 kg/km (11.7 lb/mi),2.19 L/100 km (107.4 mpg‑US)[46]
4,Antonov An-148,2004,89,"684 nmi (1,267 km)",2.89 kg/km (10.3 lb/mi),4.06 L/100 km (57.9 mpg‑US)[33]
5,Antonov An-158,2010,99,"684 nmi (1,267 km)",3 kg/km (11 lb/mi),3.79 L/100 km (62.1 mpg‑US)[33]
6,Boeing 737-300,1984,126,507 nmi (939 km),3.49 kg/km (12.4 lb/mi),3.46 L/100 km (68 mpg‑US)[47]
7,Boeing 737-600,1998,110,500 nmi (930 km),3.16 kg/km (11.2 lb/mi),3.59 L/100 km (65.5 mpg‑US)[48]
8,Boeing 737-700,1997,126,500 nmi (930 km),3.21 kg/km (11.4 lb/mi),3.19 L/100 km (74 mpg‑US)[48]
9,Boeing 737 MAX 7,2017,128,"660 nmi (1,220 km)",2.85 kg/km (10.1 lb/mi),2.77 L/100 km (84.8 mpg‑US)[46]


In [162]:
short_haul.columns = ["Model", "First flight", "Seats", "Fuel burn", "Fuel per seat"]
short_haul

,Model,First flight,Seats,Fuel burn,Fuel per seat
0,Airbus A319,1995,124,2.93 kg/km (10.4 lb/mi),2.95 L/100 km (80 mpg‑US)[58]
1,Airbus A319Neo,2015,136,2.4 kg/km (8.6 lb/mi),1.93 L/100 km (122 mpg‑US)[27]
2,Airbus A320,1987,150,3.13 kg/km (11.1 lb/mi),2.61 L/100 km (90 mpg‑US)[58]
3,Airbus A321-200,1996,180,3.61 kg/km (12.8 lb/mi),2.50 L/100 km (94 mpg‑US)[58]
4,Airbus A330-200,1997,293,5.6 kg/km (19.8 lb/mi),2.37 L/100 km (99 mpg‑US)[58]
5,Antonov An-148 (1190 nmi),2004,89,2.75 kg/km (9.8 lb/mi),3.86 L/100 km (60.9 mpg‑US)[33]
6,Antonov An-158 (1190 nmi),2010,99,2.83 kg/km (10.0 lb/mi),3.57 L/100 km (65.9 mpg‑US)[33]
7,Boeing 737-600,1998,110,2.77 kg/km (9.8 lb/mi),3.15 L/100 km (75 mpg‑US)[48]
8,Boeing 737-700,1997,126,2.82 kg/km (10.0 lb/mi),2.79 L/100 km (84 mpg‑US)[48]
9,Boeing 737-700,1997,128,2.8 kg/km (9.9 lb/mi),2.71 L/100 km (87 mpg‑US)[58]


In [163]:
medium_haul

,Model,First flight,Seats,Sector,Fuel burn,Fuel per seat
0,Airbus A320,1987,150,"2,151 nmi (3,984 km)",2.91 kg/km (10.3 lb/mi),2.43 L/100 km (97 mpg‑US)[62]
1,Airbus A321NeoLR,2016,154,"3,400 nmi (6,300 km)",2.99 kg/km (10.6 lb/mi),2.43 L/100 km (97 mpg‑US)[63]
2,Airbus A330-200,1997,241,"3,000 nmi (5,600 km)",6 kg/km (21 lb/mi),3.11 L/100 km (76 mpg‑US)[64]
3,Airbus A330-300,1992,262,"3,000 nmi (5,600 km)",6.25 kg/km (22.2 lb/mi),2.98 L/100 km (79 mpg‑US)[64]
4,Airbus A330neo-900,2016,310,"3,350 nmi (6,200 km)",6 kg/km (21 lb/mi),2.42 L/100 km (97 mpg‑US)[65]
5,Airbus A340-300,1992,262,"3,000 nmi (5,600 km)",6.81 kg/km (24.2 lb/mi),3.25 L/100 km (72 mpg‑US)[64]
6,Boeing 737 MAX-8,2017,168,"3,400 nmi (6,300 km)",2.86 kg/km (10.1 lb/mi),2.13 L/100 km (110 mpg‑US)[66]
7,Boeing 737 MAX-9,2017,144,"3,400 nmi (6,300 km)",2.91 kg/km (10.3 lb/mi),2.53 L/100 km (93 mpg‑US)[63]
8,Boeing 747-400,1988,416,"2,151 nmi (3,984 km)",10.77 kg/km (38.2 lb/mi),3.24 L/100 km (73 mpg‑US)[62]
9,Boeing 747-8,2011,467,"3,000 nmi (5,600 km)",9.9 kg/km (35 lb/mi),2.65 L/100 km (89 mpg‑US)[67]


In [164]:
long_haul

,Model,First flight,Seats,Sector,Fuel burn,Fuel per seat
0,Airbus A330-200,1997,241,"6,000 nmi (11,000 km)",6.4 kg/km (23 lb/mi),3.32 L/100 km (71 mpg‑US)[64]
1,Airbus A330neo-800,2017,248,"4,650 nmi (8,610 km)",5.45 kg/km (19.3 lb/mi),2.75 L/100 km (86 mpg‑US)[72]
2,Airbus A330neo-900,2017,300,"4,650 nmi (8,610 km)",5.94 kg/km (21.1 lb/mi),2.48 L/100 km (95 mpg‑US)[72]
3,Airbus A340-300,1992,262,"6,000 nmi (11,000 km)",7.32 kg/km (26.0 lb/mi),3.49 L/100 km (67.4 mpg‑US)[64]
4,Airbus A350-900,2013,315,"4,972 nmi (9,208 km)",6.03 kg/km (21.4 lb/mi),2.39 L/100 km (98 mpg‑US)[65]
5,Airbus A350-900,2013,315,"6,542 nmi (12,116 km)",7.07 kg/km (25.1 lb/mi),2.81 L/100 km (84 mpg‑US)[73]
6,Airbus A380,2005,525,"7,200 nmi (13,300 km)",13.78 kg/km (48.9 lb/mi),3.27 L/100 km (72 mpg‑US)[74]
7,Airbus A380,2005,544,"6,000 nmi (11,000 km)",13.78 kg/km (48.9 lb/mi),3.16 L/100 km (74 mpg‑US)[75]
8,Boeing 747-400,1988,416,"6,000 nmi (11,000 km)",11.11 kg/km (39.4 lb/mi),3.34 L/100 km (70 mpg‑US)[76]
9,Boeing 747-8,2011,467,"6,000 nmi (11,000 km)",10.54 kg/km (37.4 lb/mi),2.82 L/100 km (83 mpg‑US)[67]


# CO2 emissions

For 1kg of fuel, 3.304kg of CO2 is emitted

In [165]:
CO2_ratio = 3.304

## Keep only useful field

In [189]:
def clean_fuel_burn(x):
    return float(x.split("k")[0])

def clean_fuel_per_seat(x):
    return float(x.split("L")[0])

def clean_df(df):
    new_df = df[["Model", "Fuel burn", "Fuel per seat"]].copy()
    new_df["Fuel burn"] = [clean_fuel_burn(x) for x in new_df["Fuel burn"]]
    new_df["Fuel per seat"] = [clean_fuel_per_seat(x) for x in new_df["Fuel per seat"]]
    new_df["CO2 kg/km"] = [x*CO2_ratio for x in new_df["Fuel burn"]]
    return new_df

In [190]:
commuter_ = clean_df(commuter)
regional_ = clean_df(regional)
short_haul_ = clean_df(short_haul)
medium_haul_ = clean_df(medium_haul)
long_haul_ = clean_df(long_haul)

In [192]:
regional_.mean()

Fuel burn        2.717619
Fuel per seat    3.351667
CO2 kg/km        8.979013
dtype: float64

In [193]:
commuter_.mean()

Fuel burn        2.013333
Fuel per seat    4.683333
CO2 kg/km        6.652053
dtype: float64

In [194]:
short_haul_.mean()

Fuel burn         3.108000
Fuel per seat     2.687200
CO2 kg/km        10.268832
dtype: float64

In [195]:
medium_haul_.mean()

Fuel burn         5.582222
Fuel per seat     2.743704
CO2 kg/km        18.443662
dtype: float64

In [196]:
long_haul_.mean()

Fuel burn         8.138095
Fuel per seat     2.958095
CO2 kg/km        26.888267
dtype: float64

In [197]:
all_flights = pd.concat([commuter_, regional_, short_haul_, medium_haul_, long_haul_])

In [199]:
all_flights.mean()

Fuel burn         4.133083
Fuel per seat     3.221429
CO2 kg/km        13.655705
dtype: float64